# 减少内存位数

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                c_prec = df[col].apply(lambda x: np.finfo(x).precision).max()
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max and c_prec == np.finfo(np.float16).precision:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max and c_prec == np.finfo(np.float32).precision:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

这段代码的目的是减少DataFrame的内存占用。它首先计算DataFrame的初始内存大小，并遍历所有列，检查它们的数据类型。如果列的数据类型是整数或浮点数，它将尝试将列转换为更小的数据类型，以减少内存使用。

代码逻辑如下：

1. 定义一个列表`numerics`，包含了所有需要检查的数值数据类型。
2. 计算DataFrame `df`的初始内存使用量`start_mem`。
3. 遍历DataFrame的所有列，对于每一列：
    - 获取列的数据类型`col_type`。
    - 如果`col_type`是数值类型之一：
        - 计算列的最小值`c_min`和最大值`c_max`。
        - 如果`col_type`是整数类型：
            - 根据`c_min`和`c_max`的值，尝试将列转换为`np.int8`、`np.int16`、`np.int32`或`np.int64`中的一个更小的整数类型。
        - 如果`col_type`是浮点类型：
            - 计算列的最大精度`c_prec`。
            - 根据`c_min`、`c_max`和`c_prec`的值，尝试将列转换为`np.float16`、`np.float32`或保留为`np.float64`中的一个更小的浮点类型。
4. 计算转换后的DataFrame `df`的内存使用量`end_mem`。
5. 如果`verbose`参数为True，打印内存使用量减少的信息，包括减少的百分比。

总的来说，这个函数通过转换DataFrame中的列到更小的数据类型来减少内存占用，特别是对于包含大量数值数据的大型DataFrame，这可以带来显著的内存节省。